In [65]:
import json
from pathlib import Path
from source_p import dvr3dparser as dp

In [66]:
class CombinedInputInterface:
    # Record the commandline to run
    commands = []

    def __init__(self,inputpath):
        # Open the file read every line into a vector
        lines = open(Path(inputpath)).read().splitlines()

        # Hold the task number for filenames.
        taskcounter = 0
        linecounter = 0

        while (linecounter < len(lines)):
            line = lines[linecounter]

            # Ignore empty line
            if line=='\n':
                linecounter+=1
                continue
            # Check if is a new Fortran block
            elif line[2:9]=="Fortran":
                taskcounter+=1
                sepLine = line[10:].split(" ")

                # Optional argument:
                outname = "output{}.result".format(taskcounter)
                try:
                    # Check for optional argument
                    if len(sepLine)>2:
                        for argstr in sepLine[2:]:
                            [arg, val] = argstr.split("=")
                            if arg=="outname": outname = val
                except Exception as e:
                    print("Error reading block argument: {}".format(line))
                    raise
                    
                # Copy every line in this block to a temp file, untile next line start with &&
                with open (Path("input/temp/temptxt{}.txt".format(taskcounter)),"w+",encoding="utf-8") as f:
                    while (linecounter+1 < len(lines) and lines[linecounter+1][:2]!="&&"):
                        linecounter+=1
                        f.write(lines[linecounter]+"\n")

                
                # convert to json
                jsonpath = dp.txtToJson("input/temp/temptxt{}.txt".format(taskcounter))

                # convert to job
                with open (jsonpath) as f:
                    jsonobj = json.load(f)
                dvrparser = dp.GeneralParser("configs/{}.json".format(sepLine[0]))
                dvrparser.write(jsonobj, "input/temp/tempjob{}.job".format(taskcounter))

                # add a command to this object's command list
                self.commands.append("{} <input/temp/tempjob{}.job> {}".format(sepLine[1],taskcounter,outname))


            # If not &&Fortran, then it is a command to directly run
            else:
                # The split... is used to prevent an empty line before command block generating empty cmd in self.commands
                self.commands.append(line[2:].split('\n')[0])
            
            # Next line
            linecounter+=1
    
    def printCommands(self):
        for cmd in self.commands:
            print(cmd)

    

In [67]:
tt = CombinedInputInterface("input/combined.txt")

In [68]:
tt.printCommands()

./dvr.out <input/temp/tempjob1.job> output1_manual.result
rot.out <input/temp/tempjob2.job> output2.result
cp fort.8 fort.11
cp fort.9 fort.12

./dip.out <input/temp/tempjob3.job> output3.result
./spec.out <input/temp/tempjob4.job> output4.result


In [69]:
tt.commands

['./dvr.out <input/temp/tempjob1.job> output1_manual.result',
 'rot.out <input/temp/tempjob2.job> output2.result',
 'cp fort.8 fort.11',
 'cp fort.9 fort.12',
 '',
 './dip.out <input/temp/tempjob3.job> output3.result',
 './spec.out <input/temp/tempjob4.job> output4.result']

In [70]:
test = "test\n\n".split('\n')

In [71]:
test

['test', '', '']